#  Classification of genetic interactions  using Support Vector Machine(SVM)

Although cancer classification has improved over the past 30 years, there has been no general approach for identifying new cancer classes (class discovery) or for assigning tumors to known classes (class prediction). 
The dataset comes from a proof-of-concept study published in 1999 by Golub et al. It showed how new cases of cancer could be classified by gene expression monitoring (via DNA microarray) and thereby provided a general approach for identifying new cancer classes and assigning tumors to known classes.
The goal is to classify patients with acute myeloid leukemia (AML) and acute lymphoblastic leukemia (ALL) using the SVM algorithm.

Lets start coding :)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict 

In [ ]:

#############patients/genes########################################

data_synthetic_lethals=pd.read_excel(r'C:\Users\linigodelacruz\Documents\PhD_2018\Documentation\Calculations\Functions\version-control-functions\data-from-yeastmine-queries\data_synthetic_lethals_from_list_yeastmine.xlsx',header=2)
data_synthetic_lethals.drop(['Unnamed: 0'],axis=1)

## non synthetc lethals
data_dosage_rescue=pd.read_excel(r'C:\Users\linigodelacruz\Documents\PhD_2018\Documentation\Calculations\Functions\version-control-functions\data-from-yeastmine-queries\data_dosage_rescue_from_list_yeastmine.xlsx',header=2)
data_dosage_rescue.drop(['Unnamed: 0'],axis=1)


##################features(data to use for the prediction)###############################
##curated data on slim go terms in budding yeast SGD downloads
data_raw_slim_go=pd.read_excel(r'C:\Users\linigodelacruz\Documents\PhD_2018\Documentation\Calculations\data_sgd\slim-goterms-filtered-data.xlsx',header=0,encoding="utf-8-sig")

## published data in the cellmap.org 2016 
data_fitness_sga=pd.read_excel(r'C:\Users\linigodelacruz\Documents\PhD_2018\Documentation\Calculations\SGA-Boone-LAB\Data File S1. Raw genetic interaction datasets_ Pair-wise interaction format\Data-fitness.xlsx',header=0,sheet_name='NxN')

## curated data set on interactions in budding yeast SGD downloads
data_raw_interact=pd.read_excel(r'C:\Users\linigodelacruz\Documents\PhD_2018\Documentation\Calculations\data_sgd\interaction-filtered-data.xlsx',header=0,encoding="utf-8-sig")


data_raw_slim_go.columns=['Gene','gene-id','go-aspect','go-term','go-id','feature-type' ]
data_raw_interact.columns=['Gene', 'Interactor', 'Assay', 'Annotation', 'Notes','Phenotype','Reference-SGD','citation']
data_fitness_sga.columns=['query-strain','query-allele-name','array-strain','array-allele-name','array-type','score','p-value','query-fitness','array-fitness','double-fitness','double-fitness-std']


In [ ]:
train_data=defaultdict(dict)
target_gene='ACT1'

synt_lethals_target_gene=data_synthetic_lethals[data_synthetic_lethals['name']==target_gene]['interactor']
non_synt_lethals_target_gene=data_dosage_rescue[data_dosage_rescue['name']==target_gene]['interactor']


for i in synt_lethals_target_gene:
    counter=0
    go_per_synt_gene=data_raw_slim_go[data_raw_slim_go['Gene']==i]['go-term'].tolist()

    for j in go_per_synt_gene:
        counter=counter+1
        train_data[i][counter]=j
        train_data[i]['lethal']=1
    interactor_per_synt_gene=data_raw_interact[data_raw_interact['Gene']==i]['Interactor'].tolist()
    for k in interactor_per_synt_gene:
        counter=counter+1
        train_data[i][counter]=k
        train_data[i]['lethal']=1
    

for i in non_synt_lethals_target_gene:
    counter=0 ## when adding new rows
    go_per_non_synt_gene=data_raw_slim_go[data_raw_slim_go['Gene']==i]['go-term'].tolist()
    for j in go_per_non_synt_gene:
        counter=counter+1
        train_data[i][counter]=j
        train_data[i]['lethal']=0
    interactor_per_non_synt_gene=data_raw_interact[data_raw_interact['Gene']==i]['Interactor'].tolist()
    for k in interactor_per_non_synt_gene:
        counter=counter+1
        train_data[i][counter]=k
        train_data[i]['lethal']=0


train_data=pd.DataFrame(train_data)
train_data=train_data.T.fillna('no-info')
train_data.index=np.arange(0,len(synt_lethals_target_gene)+len(non_synt_lethals_target_gene)-1) 

train_data

After transpose, the rows have been converted to columns(7129 columns/features)



In [ ]:
train_data["patient"] = np.arange(0,len(synt_lethals_target_gene)+len(non_synt_lethals_target_gene)-1) 

Our next step is to create two variables X(matrix of independent variables) and y(vector of the dependent variable).

In [ ]:
X,y=train_data.drop(columns=['lethal']),train_data['lethal']


Next, 
We split 75% of the data into training set while 25% of the data to test set. 
The *test_size* variable is where we actually specify the proportion of the test set.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X,y,test_size = 0.25, random_state= 0)

Next step is :
to normalize the data because if we closely look at the data the range of values of independent variables varies a lot.

So when the values vary a lot in independent variables, we use **feature scaling** so that all the values remain in the comparable range.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

##########!!!!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!########## because my features are strings, dont know how to solve this

The number of columns/features that we have been working with is huge. 
We have 72 rows and 7129 columns. 
Basically we need to decrease the number of features(Dimentioanlity Reduction) to remove the possibility of [ Curse of Dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality)

For reducing the number of dimensions/features we will use the most popular dimensionality reduction algorithm i.e. **PCA(Principal Component Analysis)**.
To perform PCA we have to choose the number of features/dimensions that we want in our data.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA() 
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
total=sum(pca.explained_variance_)
k=0
current_variance=0
while current_variance/total < 0.90:
    current_variance += pca.explained_variance_[k]
    k=k+1
print(k)


The above code gives k=38.
Now let us take k=38 and apply PCA on our independent variables.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 38)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
cum_sum = pca.explained_variance_ratio_.cumsum()
cum_sum = cum_sum*100
plt.bar(range(38), cum_sum)
plt.ylabel("Cumulative Explained Variance")
plt.xlabel("Principal Components")
plt.title("Around 90% of variance is explained by the First 38 columns ")

**Note:- PCA can lead to a reduction in model performance on datasets with no or low feature correlation or does not meet the assumptions of linearity.**
The next step is to fit our data into the Support Vector Machine(SVM) algorithm but before doing that we will perform Hyperparameter optimization.

> [Hyperparameter optimization](https://en.wikipedia.org/wiki/Hyperparameter_optimization) or tuning is the problem of choosing a set of optimal hyperparameters for a learning algorithm. A hyperparameter is a parameter whose value is used to control the learning process. By contrast, the values of other parameters are learned

We will use GridSearchCV from sklearn for choosing the best hyperparameters.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
search = GridSearchCV(SVC(), parameters, n_jobs=-1, verbose=1)
search.fit(X_train, y_train)

Now check what are the best parameters for our SVM algorithm

In [ ]:
best_parameters = search.best_estimator_
print(best_parameters)

Now let's train our SVM classification model.

In [ ]:
model = SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001,
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
    
model.fit(X_train, y_train)

It is time for some predictions!

In [ ]:
y_pred=model.predict(X_test)

 Evaluating model performance

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics
print('Accuracy Score:',round(accuracy_score(y_test, y_pred),2))
#confusion matrix
cm = confusion_matrix(y_test, y_pred)


Confusion matrix and visualize it using Heatmap.

In [ ]:
class_names=[1,2,3]
fig, ax = plt.subplots()
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(y_test, y_pred)
class_names=['ALL', 'AML']

tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
sns.heatmap(pd.DataFrame(cm), annot=True, cmap="Blues" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
pd.DataFrame(cm)

Well, this example goes to show that if you just predict that every patient has AML, you’ll be correct more often than wrong.

So our SVM classification model predicted the cancer patients with 67% accuracy which is of course not that good.

What you can do is try different classifiers like Random forest, K-NN, Gradient Boosting, xgboost etc and compare the accuracies for each model.